`gpytorch` and `pytorch` has ways to make their models portable. I am not very satisfied with it becuase they still require you have `pytorch` installed. So in the library I'll write a tiny gp library that only evaluates with saved weights. 

In [1]:
gp_parameters_fname = {}
gp_parameters = {}

In [2]:
import aemulusnu_massfunction
print(aemulusnu_massfunction.__file__)

import os

package_path = os.path.dirname(aemulusnu_massfunction.__file__)
print(package_path)

/home/users/delon/aemulusnu_massfunction/aemulusnu_massfunction/__init__.py
/home/users/delon/aemulusnu_massfunction/aemulusnu_massfunction


In [3]:
import pickle

In [4]:

cosmos_f = open('../data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()

a_list_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/alist.pkl'
a_list_f = open(a_list_fname, 'rb')
a_list = pickle.load(a_list_f)
a_list_f.close()

In [5]:
from tqdm import tqdm

In [6]:
key_ordering = ['10^9 As', 'ns', 'H0', 'w0', 'ombh2', 'omch2', 'nu_mass_ev']

In [7]:
from aemulusnu_hmf.utils import *


In [8]:
weird_boxes = []


errors = {a:{} for a in a_list}
X = []
Y = []
z_to_a = {}
a_to_z = {}
kt = np.logspace(-3, 1, 100) # h/Mpc
for box in tqdm(cosmo_params):
    if(box in weird_boxes):
        continue
    curr_cosmo = cosmo_params[box]
    if(box == 'Box_n50_0_1400'):
        print(curr_cosmo)

    curr_cosmo_values = [curr_cosmo[curr_key] for curr_key in key_ordering]

    h = curr_cosmo['H0']/100

    Ωb =  curr_cosmo['ombh2'] / h**2
    Ωc =  curr_cosmo['omch2'] / h**2



    try:
        with open("/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/%s_params.pkl"%(box), "rb") as f:
            MLE_params = pickle.load(f)
            param_values = list(MLE_params.values())
            Y+= [param_values]
            X+= [curr_cosmo_values]
    except:
        print(box, z)
    for a in a_list:
        z = scaleToRedshift(a)
        z_to_a[z] = a
        a_to_z[a] = z

 69%|██████▊   | 103/150 [00:02<00:01, 36.86it/s]

{'ns': 0.97000003, 'H0': 67.0, 'w0': -1.0, 'ombh2': 0.0223, 'omch2': 0.12, 'nu_mass_ev': 0.07071068, '10^9 As': 2.10100315}


100%|██████████| 150/150 [00:04<00:00, 32.11it/s]


In [9]:
import pickle
with open(package_path + "/emulator.pkl", "rb") as f:
    model, in_scaler, out_scaler, likelihood = pickle.load(f)

In [10]:
X = np.array(X)
Y = np.array(Y)

In [11]:
transX = in_scaler.transform(X)
transY = out_scaler.transform(Y)

In [12]:
model.mean_module.base_means[0].__dict__

{'training': True,
 '_parameters': OrderedDict([('weights',
               Parameter containing:
               tensor([[-0.2005],
                       [-0.0838],
                       [ 0.0316],
                       [ 0.0549],
                       [ 0.0687],
                       [-0.3498],
                       [ 0.0295]], requires_grad=True)),
              ('bias',
               Parameter containing:
               tensor([2.3503], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict([(2,
               <torch.nn.modules.module._WrappedHook at 0x

In [13]:
import aemulusnu_hmf
from aemulusnu_hmf.gaussian_process import *


In [14]:
hmf_lib_package_path = os.path.dirname(aemulusnu_hmf.__file__)
print(hmf_lib_package_path)

/home/users/delon/aemulusnu_hmf_lib/aemulusnu_hmf


In [15]:
import numpy as np

In [16]:
model.mean_module.base_means

ModuleList(
  (0-7): 8 x LinearMean()
)

In [17]:
model.mean_module.base_means[0].weights.detach().numpy()

array([[-0.20054933],
       [-0.0838212 ],
       [ 0.03163494],
       [ 0.0549029 ],
       [ 0.06872709],
       [-0.34980562],
       [ 0.02951493]], dtype=float32)

In [18]:
in_scaler.__dict__['min_val']

array([ 1.11197940e+00,  9.30572150e-01,  5.21808281e+01, -1.55389726e+00,
        1.73423700e-02,  8.05167200e-02,  1.01726600e-02])

In [19]:
#save rawX, trainX trainY, inscaler params
local_fname = hmf_lib_package_path+'/data/train_x.txt'
np.savetxt(local_fname, transX)

local_fname = hmf_lib_package_path+'/data/raw_x.txt'
np.savetxt(local_fname, X)


local_fname = hmf_lib_package_path+'/data/train_y.txt'
np.savetxt(local_fname, Y)



local_fname = hmf_lib_package_path+'/data/in_normalizer_min_vals.txt'
np.savetxt(local_fname, in_scaler.__dict__['min_val'])


local_fname = hmf_lib_package_path+'/data/in_normalizer_max_vals.txt'
np.savetxt(local_fname, in_scaler.__dict__['max_val'])


In [20]:
n_means = len(model.mean_module.base_means)
gp_parameters_fname['mean_module'] = {}
gp_parameters_fname['mean_module']['base_means'] = [{} for i in range(n_means)] 
for i in range(n_means):
    gp_parameters_fname['mean_module']['base_means'][i]['weights'] = 'data/mean_module.base_means.%d.weights.txt'%i
    local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['mean_module']['base_means'][i]['weights']
    np.savetxt(local_fname, 
              model.mean_module.base_means[i].weights.detach().numpy())
    gp_parameters_fname['mean_module']['base_means'][i]['bias'] = 'data/mean_module.base_means.%d.bias.txt'%i
    local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['mean_module']['base_means'][i]['bias']
    np.savetxt(local_fname, 
              model.mean_module.base_means[i].bias.detach().numpy())

In [21]:
from pkg_resources import resource_filename


In [22]:
base_means = []
for i in range(n_means):
    curr_weights = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['mean_module']['base_means'][i]['weights']))
    curr_bias    = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['mean_module']['base_means'][i]['bias']))
    base_means += [LinearMean(curr_weights,
                              curr_bias)]

In [23]:
base_means

In [24]:
mean_module = MultitaskMean(base_means)

In [25]:
mean_module(transX).shape

(150, 8)

In [26]:
model.state_dict()

OrderedDict([('likelihood.raw_noise', tensor([inf])),
             ('likelihood.raw_noise_constraint.lower_bound', tensor(-inf)),
             ('likelihood.raw_noise_constraint.upper_bound',
              tensor(1.0000e-08)),
             ('mean_module.base_means.0.weights',
              tensor([[-0.2005],
                      [-0.0838],
                      [ 0.0316],
                      [ 0.0549],
                      [ 0.0687],
                      [-0.3498],
                      [ 0.0295]])),
             ('mean_module.base_means.0.bias', tensor([2.3503])),
             ('mean_module.base_means.1.weights',
              tensor([[-0.0323],
                      [ 0.0252],
                      [-0.0070],
                      [ 0.1526],
                      [ 0.0469],
                      [ 0.0129],
                      [-0.0088]])),
             ('mean_module.base_means.1.bias', tensor([-0.0974])),
             ('mean_module.base_means.2.weights',
              tensor([[

In [27]:
model.covar_module.data_covar_module.kernels[0].base_kernel.lengthscale.detach().numpy()

array([[2.2953115, 2.5196962, 2.2107248, 2.0373337, 3.4636915, 2.0017018,
        2.7992213]], dtype=float32)

In [28]:
gp_parameters_fname['covar_module'] = {}
gp_parameters_fname['covar_module']['data_covar_module'] = {}
gp_parameters_fname['covar_module']['data_covar_module']['matern'] = {}

gp_parameters_fname['covar_module']['data_covar_module']['matern']['nu'] = 'data/covar_module.data_covar_module.matern.nu.txt'
local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['covar_module']['data_covar_module']['matern']['nu']
np.savetxt(local_fname, 
           [model.covar_module.data_covar_module.kernels[0].base_kernel.nu])


gp_parameters_fname['covar_module']['data_covar_module']['matern']['lengthscale'] = 'data/covar_module.data_covar_module.matern.lengthscale.txt'
local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['covar_module']['data_covar_module']['matern']['lengthscale']
np.savetxt(local_fname, 
           model.covar_module.data_covar_module.kernels[0].base_kernel.lengthscale.detach().numpy())

In [29]:
model.covar_module.data_covar_module.kernels[0].base_kernel

MaternKernel(
  (raw_lengthscale_constraint): Positive()
)

In [30]:
nu = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['covar_module']['data_covar_module']['matern']['nu']))
lengthscale = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['covar_module']['data_covar_module']['matern']['lengthscale']))
MaternKernel(nu, lengthscale)(transX, transX).shape

(150, 150)

In [31]:
model.covar_module.data_covar_module.kernels[0].outputscale.item()

0.03372954577207565

In [32]:
gp_parameters_fname['covar_module']['data_covar_module']['scale'] = 'data/covar_module.data_covar_module.scale.txt'
local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['covar_module']['data_covar_module']['scale']
np.savetxt(local_fname, 
           [model.covar_module.data_covar_module.kernels[0].outputscale.item()])

In [33]:
scale = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['covar_module']['data_covar_module']['scale']))
ScaleKernel(MaternKernel(nu, lengthscale), scale)

In [34]:
gp_parameters_fname['covar_module']['data_covar_module']['constant'] = 'data/covar_module.data_covar_module.constant.txt'
local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['covar_module']['data_covar_module']['constant']
np.savetxt(local_fname, 
           [model.covar_module.data_covar_module.kernels[1].constant.item()])

In [35]:
constant = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['covar_module']['data_covar_module']['constant']))
ConstantKernel(constant)(transX, transX).shape

()

In [36]:
AdditiveKernel(ScaleKernel(MaternKernel(nu, lengthscale), scale), ConstantKernel(constant))

In [37]:
model.covar_module.task_covar_module.var.detach().numpy()

array([0.68222463, 0.22824858, 0.0447395 , 0.09474372, 0.00587697,
       0.01507122, 0.02656781, 0.01408685], dtype=float32)

In [38]:
gp_parameters_fname['covar_module']['task_covar_module'] = {}

In [39]:
gp_parameters_fname['covar_module']['task_covar_module']['covar_factor'] = 'data/covar_module.task_covar_module.covar_factor.txt'
local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['covar_module']['task_covar_module']['covar_factor'] 
np.savetxt(local_fname, 
           model.covar_module.task_covar_module.covar_factor.detach().numpy())

In [40]:
gp_parameters_fname['covar_module']['task_covar_module']['var'] = 'data/covar_module.task_covar_module.var.txt'
local_fname = hmf_lib_package_path+'/'+gp_parameters_fname['covar_module']['task_covar_module']['var'] 
np.savetxt(local_fname, 
           model.covar_module.task_covar_module.var.detach().numpy())

In [41]:
covar_factor = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['covar_module']['task_covar_module']['covar_factor']))
var = np.loadtxt(resource_filename('aemulusnu_hmf', gp_parameters_fname['covar_module']['task_covar_module']['var']))
IndexKernel(covar_factor, var)

In [42]:
covar_module = MultitaskKernel(task_covar_module = IndexKernel(covar_factor, var),
                data_covar_module = AdditiveKernel(ScaleKernel(MaternKernel(nu, lengthscale), scale), ConstantKernel(constant)))

In [43]:
covar_module(transX, transX).shape

(1200, 1200)

In [44]:
import scipy

In [45]:
train_x = transX
test_x = transX
train_y = transY
KXSX = covar_module(test_x, train_x)
KXSXS = covar_module(test_x, test_x)
KXXS = covar_module(train_x, test_x)
KXX = covar_module(train_x, train_x)

In [46]:
L = np.linalg.cholesky(KXX)

_y = train_y - mean_module(train_x)
_y = _y.flatten()

alpha = scipy.linalg.solve_triangular(L.T, scipy.linalg.solve_triangular(L, _y, lower=True))

my_predictive_mean = np.reshape(np.einsum('ij,j->i', KXSX, alpha), (len(test_x), len(train_y[0])))+ mean_module(test_x)

In [47]:
out_scaler.inverse_transform(my_predictive_mean)

array([[ 2.00484397,  0.01516664,  1.10048519, ...,  0.17710185,
         1.24197726,  0.06490117],
       [ 2.01523049, -0.05461264,  1.13079773, ...,  0.23289802,
         1.22912063,  0.03383619],
       [ 2.1398625 ,  0.01624387,  0.99733895, ...,  0.20665823,
         1.22758707,  0.05386155],
       ...,
       [ 2.00951695,  0.03799582,  1.04211064, ...,  0.19116918,
         1.22295621,  0.03435663],
       [ 2.07722698, -0.06035875,  1.11525458, ...,  0.21065105,
         1.24119778,  0.02658227],
       [ 1.96833471,  0.01042105,  1.08300666, ...,  0.21059359,
         1.21321417,  0.05628117]])

In [48]:
my_predictive_mean

array([[ 2.00484397,  0.01516664,  1.10048519, ...,  0.17710185,
         1.24197726,  0.06490117],
       [ 2.01523049, -0.05461264,  1.13079773, ...,  0.23289802,
         1.22912063,  0.03383619],
       [ 2.1398625 ,  0.01624387,  0.99733895, ...,  0.20665823,
         1.22758707,  0.05386155],
       ...,
       [ 2.00951695,  0.03799582,  1.04211064, ...,  0.19116918,
         1.22295621,  0.03435663],
       [ 2.07722698, -0.06035875,  1.11525458, ...,  0.21065105,
         1.24119778,  0.02658227],
       [ 1.96833471,  0.01042105,  1.08300666, ...,  0.21059359,
         1.21321417,  0.05628117]])

In [53]:
np.allclose(my_predictive_mean, Y, atol=0, rtol=1e-8)

True

In [50]:
gp_parameters_fname

{'mean_module': {'base_means': [{'weights': 'data/mean_module.base_means.0.weights.txt',
    'bias': 'data/mean_module.base_means.0.bias.txt'},
   {'weights': 'data/mean_module.base_means.1.weights.txt',
    'bias': 'data/mean_module.base_means.1.bias.txt'},
   {'weights': 'data/mean_module.base_means.2.weights.txt',
    'bias': 'data/mean_module.base_means.2.bias.txt'},
   {'weights': 'data/mean_module.base_means.3.weights.txt',
    'bias': 'data/mean_module.base_means.3.bias.txt'},
   {'weights': 'data/mean_module.base_means.4.weights.txt',
    'bias': 'data/mean_module.base_means.4.bias.txt'},
   {'weights': 'data/mean_module.base_means.5.weights.txt',
    'bias': 'data/mean_module.base_means.5.bias.txt'},
   {'weights': 'data/mean_module.base_means.6.weights.txt',
    'bias': 'data/mean_module.base_means.6.bias.txt'},
   {'weights': 'data/mean_module.base_means.7.weights.txt',
    'bias': 'data/mean_module.base_means.7.bias.txt'}]},
 'covar_module': {'data_covar_module': {'matern':

In [51]:
my_predictive_mean[100]

array([ 1.96331446,  0.01337108,  1.11043021, -0.08513964,  0.40331857,
        0.20879524,  1.21787705,  0.03729562])

In [54]:
cosmo_params

{'Box0_1400': {'ns': 1.0094583,
  'H0': 59.729599,
  'w0': -1.35573566,
  'ombh2': 0.0174071,
  'omch2': 0.13311985,
  'nu_mass_ev': 0.03654721,
  '10^9 As': 1.47405133},
 'Box1_1400': {'ns': 0.95570517,
  'H0': 79.06343842,
  'w0': -0.91724169,
  'ombh2': 0.01734237,
  'omch2': 0.14155602,
  'nu_mass_ev': 0.11126766,
  '10^9 As': 1.47786383},
 'Box2_1400': {'ns': 0.94857067,
  'H0': 72.40898132,
  'w0': -1.25838923,
  'ombh2': 0.02648635,
  'omch2': 0.13412274,
  'nu_mass_ev': 0.18273698,
  '10^9 As': 1.67653236},
 'Box3_1400': {'ns': 0.96076578,
  'H0': 63.68556213,
  'w0': -1.2117964,
  'ombh2': 0.01962075,
  'omch2': 0.11820062,
  'nu_mass_ev': 0.02462526,
  '10^9 As': 1.74438897},
 'Box4_1400': {'ns': 0.98394543,
  'H0': 71.57246399,
  'w0': -0.7596882,
  'ombh2': 0.01848138,
  'omch2': 0.11300255,
  'nu_mass_ev': 0.23948339,
  '10^9 As': 1.66055303},
 'Box5_1400': {'ns': 0.96076578,
  'H0': 63.68556213,
  'w0': -1.2117964,
  'ombh2': 0.01962075,
  'omch2': 0.11820062,
  'nu_mass_